*This notebook is intended to accompany the Cytiva UNICORN Tetra Data Workshop in TetraU. Specific references to Labels and sample data can be substituted for other ÄKTA data.*

# Configure Python Imports

In order to run our Jupyter Notebook,  we will need the following libraries:
- requests:  This library is used to make HTTP Requests to the TDP API
- json:  Allows us to manipulate files as JSON Objects
- pandas:  This is a very useful library for storing data in tabular structures
- numpy:  Open Source Framework for mathematical computation
- matplotlib:  Library for creating visualizations of your data

In [ ]:
import requests, json, pandas, numpy, matplotlib.pyplot as plt
%matplotlib inline

# Configure Connection Variables

Create and store information on how to connect to the TDP API. Set the following values in the code cell below, inside the empty quotation marks:

* Set `tdpHostname` to point at the correct URL for your TDP environment (such as "platform.tetrascience.com" without a "https://" or the forward slash at the end)
* Set `orgSlug` to be your org slug (ex: `training-jsmith`)
* Set `userToken` to an auth token, either your user's personal JWT or a Service User's token
 * To get your user's JWT value, open or return to a Chrome tab with TDP, click on the main hamburger menu in the top left corner and select My Account. You should be presented with the Account details screen. Click the "Copy Token" button. Your clipboard will now have the token needed for calling the API.
* Set `userLabel` to match the value that you had entered in Lab 1 for your “user” Label. It should look similar to the following: [first_name]-[last_name]. E.g. if your name is John Smith, you would have used `john-smith`. Verify this value on a file or in the Agent configuration if you do not get results in later steps.

In [ ]:
# Example hostnames: "platform.tetrascience.com" "tetrascience-uat.com"
tdpHostname = ""
orgSlug = ""
userToken = ""
userLabel = ""

headers = {"x-org-slug": orgSlug, "ts-auth-token": userToken}
apiRoot = "https://api." + tdpHostname
SearchURL = apiRoot + "/v1/datalake/searchEql"

SearchURL

# Set Query to Search for All UNICORN System Names

The TDP API uses ElasticSearch for indexing the UNICORN content. This powerful tool allows advanced searching against the data to find the appropriate information based upon your use case(s).

In this scenario, we are creating a query to find all of the system names for the UNICORN data in the organization specified earlier.  The payload has three components:
* Including `"size": 0` causes only the aggregation rusults to be returned, and not the search results themselves
* The "query" section allows filtering, and we have 2 conditions:
  * We only want IDS data in the "akta" schema, since there may be other IDS schemas with the "data.system.name" field
  * We only want data from files which have the "user" label set during this workshop, requiring the "nested" query to check the label name and value
* The "aggs" body includes a name for the aggregation so it can be referenced later in the notebook, the field to aggregate, and the maximum number of unique values to return.

For more detailed information on aggregations using Elasticsearch: https://www.elastic.co/guide/en/elasticsearch/reference/current/search-aggregations.html

In [ ]:
payload = {
  "size": 0,
  "query": {
    "bool": {
      "must": [
        {
          "term": {"idsType": "akta"}
        },
        {
          "nested": {
            "path": "labels",
            "query": {
              "bool": {
                "must": [
                  {"term": {"labels.name": "user"}},
                  {"term": {"labels.value": userLabel}}
                ]
              }
            }
          }
        }
      ]
    }
  },
  "aggs": {
    "unique_system_names": {
      "terms": { "field": "data.system.name", "size": 500 }
    }
  }
}

# Run Search Request and Display Results

We will now use the requests library to make a request to the TDP API. We have previously configured the connection variables as well as the query we are executing.

Our final line just has the variable "result" in it which tells the Notebook to print the value of the variable.

In [ ]:
request = requests.post(SearchURL, json=payload, headers=headers)

result = request.json()

result

Now let's isolate the results by accessing data by attribute names and assign it to a Pandas DataFrame object for viewing the data as a table. The sample data only came from 2 systems, but there could be many more results depending on the number of instruments and Agents.

We are also renaming the columns in the DataFrame to make them more human readable.

In [ ]:
systems = result.get("aggregations").get("unique_system_names").get("buckets")

df = pandas.DataFrame(systems)
df = df.rename(columns={"key": 'System Name', "doc_count": "No Results"})

df

# Get All Results for the Systems and Display in a Table

First let's create the payload for the API Query. We are retrieving values from the first two rows of the previous data table to pass in to the query, which should be the only rows.

The "payload_sys" has several parts:
* The "size" specifies the number of results we desire. This is the sum of the number of results from earlier.
* The "query" body has two main parts:
  * The "must" array block is a logical AND so all conditions in it must be true. Like the aggregate query earlier, we only want results in the "akta" schema, and with the appropriate "user" label.
  * The "should" array block is a logical OR, so one or more conditions (based on "minimum_should_match" set below) should be true to be. We use this to include results associated with either of the System Names in the DataFrame built from the previous request.

The search request with this payload is sent, and the hits returned are normalized into two DataFrame objects, which are joined to create the full dataset:
* "df_peaks" contains the nested "peaks" data from each IDS file, and several important non-nested fields in the "meta" block
* "df_columns" contains the nested "columns" data from each IDS file

The "Fields" variable is being used for specifying the data of interest.  Notice how we use the json_normalize function to allow us to specify data in nested JSON objects.  We are also renaming the JSON paths to field names that are more human readable in the DataFrame.

The derived column "Peak End" is created by adding the values in the "Peak Start" and "Peak Width" columns. This is added to the DataFrame, and "Peak Width" is dropped from the DataFrame.

You'll notice that we are using the pandas DataFrame "fillna" function.  This allows us to replace Null values with something of our choosing.

Lastly, we will sort the results by the Result Name and Peak Number.

In [ ]:
payload_sys = {
  "size": int(df["No Results"].sum()),
  "query": {
    "bool": {
      "must": [
        {
          "term": {"idsType": "akta"}
        },
        {"nested": {
          "path": "labels",
          "query": {
            "bool": {
              "must": [
                {"term": {"labels.name": "user"}},
                {"term": {"labels.value": userLabel}}
              ]
            }
          }
        }
      }
    ],
    "should": [
      {"term": {"data.system.name": df["System Name"][0]}},
      {"term": {"data.system.name": df["System Name"][1]}}
    ],
    "minimum_should_match" : 1,
    }
  }
}

request_sys = requests.post(SearchURL, json=payload_sys, headers=headers)
result_sys = request_sys.json()

allHits = result_sys.get("hits").get("hits")

df_peaks = pandas.json_normalize(
  allHits,
  ["_source", ["data", "result", "peaks"]],
  record_prefix='_source.data.result.peaks.',
  meta=[["_source", "fileId"]
        , ["_source", "data", "system", "name"]
        , ["_source", "data", "method", "instrument", "name"]
        , ["_source", "data", "result", "name"]
        , ["_source", "data", "result", "created_at"]])

df_columns = pandas.json_normalize(
  allHits,
  ["_source", ["data", "method", "instrument", "columns"]],
  record_prefix='_source.data.method.instrument.columns.')

df_all = df_peaks.join(df_columns)

Fields = ["_source.fileId"
          , "_source.data.system.name"
          , "_source.data.method.instrument.name"
          , "_source.data.method.instrument.columns.name"
          , "_source.data.result.name"
          , "_source.data.result.created_at"
          , "_source.data.result.peaks.number"
          , "_source.data.result.peaks.name"
          , "_source.data.result.peaks.area.value"
          , "_source.data.result.peaks.area.unit"
          , "_source.data.result.peaks.retention_volume.value"
          , "_source.data.result.peaks.retention_volume.unit"
          , "_source.data.result.peaks.start.value"
          , "_source.data.result.peaks.start.unit"
          , "_source.data.result.peaks.width.value"]

renamedDataFrame = df_all[Fields].rename(
    columns={"_source.fileId": "File Id"
             , "_source.data.system.name": "System Name"
             , "_source.data.method.instrument.name": "Instrument Name"
             , "_source.data.method.instrument.columns.name": "Column Name"
             , "_source.data.result.name": "Result Name"
             , "_source.data.result.created_at": "File Creation Time"
             , "_source.data.result.peaks.number": "Peak Number"
             , "_source.data.result.peaks.name": "Peak Name"
             , "_source.data.result.peaks.area.value": "Peak Area Value"
             , "_source.data.result.peaks.area.unit": "Peak Area Unit"
             , "_source.data.result.peaks.retention_volume.value": "Retention Volume Value"
             , "_source.data.result.peaks.retention_volume.unit": "Retention Volume Unit"
             , "_source.data.result.peaks.start.value": "Peak Start Value"
             , "_source.data.result.peaks.start.unit": "Peak Start Unit"
             , "_source.data.result.peaks.width.value": "Peak Width Value"})

renamedDataFrame["Peak End Value"] = renamedDataFrame["Peak Start Value"] + renamedDataFrame["Peak Width Value"]
renamedDataFrame.drop("Peak Width Value", axis=1, inplace=True)

renamedDataFrame["Column Name"].fillna("Missing Value", inplace=True)
renamedDataFrame.sort_values(by=["Result Name", "Peak Number"], inplace=True)
renamedDataFrame

# Display a Chart Showing the Peak Area

Let's compare the results from the 3 runs with similar names starting with "190213 5mL ProA EluteDesalt with Buffer change" by graphing the Peak Area of those results. We will use a scatter plot to easily tell if the data points are identical (fully overlapping) or different (not touching or with a partial overlap).

We know there will be 3 runs in the data set, so the `markers` array is defined explicitly with 3 different shapes. The marker styles are documented here: https://matplotlib.org/stable/api/markers_api.html

`mCount` is used to ensure that if there is unexpectedly more data, the marker shapes will be reused instead of throwing an exception.

Then, we create a new `comparison_df` DataFrame by selecting only those rows which have a "Result Name" starting with the value seen in the results table "Result Name" column, without the ellipsis (`...`) at the end.

The units for the Retention Volume and Peak Area should be consistent across all rows of the DataFrame, so the first (and only) unique value is selected and saved for reference later.

Next, the results are grouped by the Result Name so each run will have its data plotted as a series. For each group, the Peak Area is plotted, and shown with a different color (assigned automatically) and shape (based on the chosen markers).

Due to the length of the result name, the legend is set outside of the chart, and the labels are set, including the units for each axis based on the variables populated from the DataFrame.

In [ ]:
fig, ax = plt.subplots();

markers = ["o", "^", "P"]
mCount = 0

comparison_df = renamedDataFrame[renamedDataFrame["Result Name"]
                                 .str.startswith("190213 5mL ProA EluteDesalt with Buffer change")]

retentionVolUnit = comparison_df["Retention Volume Unit"].unique()[0]
peakAreaUnit = comparison_df["Peak Area Unit"].unique()[0]

for name, grp in comparison_df.groupby("Result Name"):
    ax.plot(grp["Retention Volume Value"], grp["Peak Area Value"], marker=markers[mCount], linestyle="", ms=12, label=name)
    mCount += 1
    if mCount == len(markers) :
      # Reuse markers if more data is plotted
      mCount = 0

ax.legend(bbox_to_anchor=(1.04, 1), loc="upper left")
ax.set_xlabel(f"Retention Volume ({retentionVolUnit})")
ax.set_ylabel(f"Peak Area ({peakAreaUnit})")

plt.show()

Note that 2 of the runs (ending in "005" and "006") have the same peak area and retention volume values, so their plot points overlap exactly. Depending on the colors and shape of the markers, they may not all be visible in the chart.

But the other run (ending in "003") has slightly lower values for Peak 1 and Peak 3. You can check the exact values in the original table to determine how much the difference is for each peak, and possibly continue analyzing the data to identify why this occurred.